## IMPORTS

In [2]:
import numpy as np
import tifffile as tiff
from skimage.morphology import skeletonize  # Import for skeletonization
from skan import Skeleton, summarize
from joblib import Parallel, delayed
import sys
import cc3d

sys.path.append('C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24')
from cleaning_functions.cleaning_functions import neighbors, is_junction_point, remove_branches, print_branches
from cleaning_functions.connecting_functions import connect_blobs_3d

## BOTTOM - GT

In [4]:
binary_volume = tiff.imread("C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/FINAL/skeleleton_clean_x20_th5_BOTTOM.tif")

skel = Skeleton(binary_volume)
branch_data = summarize(skel)

labels_out = cc3d.connected_components(binary_volume)
labels_out, N = cc3d.connected_components(binary_volume, return_N=True)
print(f'Number of LAbels: {N}')

# Initialize a dictionary to map each branch ID to its corresponding blob(s)
branch_blob_mapping = {}
pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

# Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
for branch_id in branch_data.index:
    # Get the coordinates of the current branch in the skeleton structure
    coordinates = skel.path_coordinates(branch_id)
    
    # Fetch the labels at those coordinates from the labels_out array
    branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
    
    # Find the unique label(s) associated with this branch
    unique_blob_labels = np.unique(branch_labels)
    
    # Store the unique label(s) in the branch_blob_mapping
    if len(unique_blob_labels) > 1:
        branch_blob_mapping[branch_id] = unique_blob_labels
    else:
        branch_blob_mapping[branch_id] = unique_blob_labels[0]
    
    # Step 3: Map each pixel in this branch to its corresponding blob label(s)
    for i, coord in enumerate(coordinates):
        pixel_blob_mapping[tuple(coord)] = branch_labels[i]


binary_volume_copy = np.copy(binary_volume)
while True:
    # Paso 4: Repetir los pasos 2 y 3 hasta que todos los blobs estén conectados
    binary_volume_copy = connect_blobs_3d(labels_out, pixel_blob_mapping, binary_volume_copy)
    
    # Free up memory by deleting unused variables
    del labels_out, pixel_blob_mapping, branch_blob_mapping
    
    labels_out, N = cc3d.connected_components(binary_volume_copy, return_N=True)
    print(f"Number of blobs: {N}")
    
    # Reinitialize pixel_blob_mapping to reduce memory usage
    # Initialize a dictionary to map each branch ID to its corresponding blob(s)
    branch_blob_mapping = {}
    pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

    skel = Skeleton(binary_volume_copy)
    branch_data = summarize(skel)

    # Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
    for branch_id in branch_data.index:
        # Get the coordinates of the current branch in the skeleton structure
        coordinates = skel.path_coordinates(branch_id)
        
        # Fetch the labels at those coordinates from the labels_out array
        branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
        
        # Find the unique label(s) associated with this branch
        unique_blob_labels = np.unique(branch_labels)
        
        # Store the unique label(s) in the branch_blob_mapping
        if len(unique_blob_labels) > 1:
            branch_blob_mapping[branch_id] = unique_blob_labels
        else:
            branch_blob_mapping[branch_id] = unique_blob_labels[0]
        
        # Step 3: Map each pixel in this branch to its corresponding blob label(s)
        for i, coord in enumerate(coordinates):
            pixel_blob_mapping[tuple(coord)] = branch_labels[i]
    
    if N == 2:
        print("All blobs are connected!")
        break

tiff.imwrite('C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/connected_blobs_BOTTOM.tif', binary_volume_copy)


Number of LAbels: 32
Connecting blobs 15 and 17
Closest points: [54 68 55] in blob 15, [56 67 54] in blob 17
Number of blobs: 31
Connecting blobs 7 and 8
Closest points: [ 37 117  20] in blob 7, [ 39 115  21] in blob 8
Number of blobs: 30
Connecting blobs 28 and 29
Closest points: [94 14 91] in blob 28, [96 13 91] in blob 29
Number of blobs: 29
Connecting blobs 25 and 27
Closest points: [89 11 99] in blob 25, [90 13 96] in blob 27
Number of blobs: 28
Connecting blobs 9 and 10
Closest points: [ 44 107  24] in blob 9, [ 44 105  25] in blob 10
Number of blobs: 27
Connecting blobs 5 and 7
Closest points: [ 36 119  19] in blob 5, [ 37 117  20] in blob 7
Number of blobs: 26
Connecting blobs 5 and 7
Closest points: [ 39 115  21] in blob 5, [ 40 113  22] in blob 7
Number of blobs: 25
Connecting blobs 9 and 10
Closest points: [ 48 100  26] in blob 9, [50 98 27] in blob 10
Number of blobs: 24
Connecting blobs 17 and 18
Closest points: [76 51 61] in blob 17, [76 52 59] in blob 18
Number of blobs:

## BOTTOM - UNET

In [3]:
binary_volume = tiff.imread("C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/FINAL/skeleleton_clean_x20_th5_UNET_BOTTOM.tif")

skel = Skeleton(binary_volume)
branch_data = summarize(skel)

labels_out = cc3d.connected_components(binary_volume)
labels_out, N = cc3d.connected_components(binary_volume, return_N=True)
print(f'Number of LAbels: {N}')

# Initialize a dictionary to map each branch ID to its corresponding blob(s)
branch_blob_mapping = {}
pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

# Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
for branch_id in branch_data.index:
    # Get the coordinates of the current branch in the skeleton structure
    coordinates = skel.path_coordinates(branch_id)
    
    # Fetch the labels at those coordinates from the labels_out array
    branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
    
    # Find the unique label(s) associated with this branch
    unique_blob_labels = np.unique(branch_labels)
    
    # Store the unique label(s) in the branch_blob_mapping
    if len(unique_blob_labels) > 1:
        branch_blob_mapping[branch_id] = unique_blob_labels
    else:
        branch_blob_mapping[branch_id] = unique_blob_labels[0]
    
    # Step 3: Map each pixel in this branch to its corresponding blob label(s)
    for i, coord in enumerate(coordinates):
        pixel_blob_mapping[tuple(coord)] = branch_labels[i]


binary_volume_copy = np.copy(binary_volume)
while True:
    # Paso 4: Repetir los pasos 2 y 3 hasta que todos los blobs estén conectados
    binary_volume_copy = connect_blobs_3d(labels_out, pixel_blob_mapping, binary_volume_copy)
    
    # Free up memory by deleting unused variables
    del labels_out, pixel_blob_mapping, branch_blob_mapping
    
    labels_out, N = cc3d.connected_components(binary_volume_copy, return_N=True)
    print(f"Number of blobs: {N}")
    
    # Reinitialize pixel_blob_mapping to reduce memory usage
    # Initialize a dictionary to map each branch ID to its corresponding blob(s)
    branch_blob_mapping = {}
    pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

    skel = Skeleton(binary_volume_copy)
    branch_data = summarize(skel)

    # Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
    for branch_id in branch_data.index:
        # Get the coordinates of the current branch in the skeleton structure
        coordinates = skel.path_coordinates(branch_id)
        
        # Fetch the labels at those coordinates from the labels_out array
        branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
        
        # Find the unique label(s) associated with this branch
        unique_blob_labels = np.unique(branch_labels)
        
        # Store the unique label(s) in the branch_blob_mapping
        if len(unique_blob_labels) > 1:
            branch_blob_mapping[branch_id] = unique_blob_labels
        else:
            branch_blob_mapping[branch_id] = unique_blob_labels[0]
        
        # Step 3: Map each pixel in this branch to its corresponding blob label(s)
        for i, coord in enumerate(coordinates):
            pixel_blob_mapping[tuple(coord)] = branch_labels[i]
    
    if N == 16:
        print("All blobs are connected!")
        break

tiff.imwrite('C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/connected_blobs_UNET_BOTTOM.tif', binary_volume_copy)


Number of LAbels: 103
Connecting blobs 95 and 96
Closest points: [134  68 141] in blob 95, [134  70 141] in blob 96
Number of blobs: 102
Connecting blobs 57 and 58
Closest points: [ 85 132 117] in blob 57, [ 85 134 116] in blob 58
Number of blobs: 101
Connecting blobs 33 and 34
Closest points: [ 68 140  88] in blob 33, [ 69 140  86] in blob 34
Number of blobs: 100
Connecting blobs 68 and 70
Closest points: [ 93  41 102] in blob 68, [ 94  42 104] in blob 70
Number of blobs: 99
Connecting blobs 98 and 99
Closest points: [157   1 195] in blob 98, [160   1 195] in blob 99
Number of blobs: 98
Connecting blobs 51 and 56
Closest points: [ 82 132 118] in blob 51, [ 85 132 117] in blob 56
Number of blobs: 97
Connecting blobs 76 and 78
Closest points: [106  18  80] in blob 76, [108  20  78] in blob 78
Number of blobs: 96
Connecting blobs 33 and 37
Closest points: [ 68 142  88] in blob 33, [ 70 143  88] in blob 37
Number of blobs: 95
Connecting blobs 62 and 66
Closest points: [93 46 99] in blob 6

## TOP - GT

In [4]:
binary_volume = tiff.imread("C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/FINAL/skeleleton_clean_x20_th5_TOP.tif")

skel = Skeleton(binary_volume)
branch_data = summarize(skel)

labels_out = cc3d.connected_components(binary_volume)
labels_out, N = cc3d.connected_components(binary_volume, return_N=True)
print(f'Number of LAbels: {N}')

# Initialize a dictionary to map each branch ID to its corresponding blob(s)
branch_blob_mapping = {}
pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

# Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
for branch_id in branch_data.index:
    # Get the coordinates of the current branch in the skeleton structure
    coordinates = skel.path_coordinates(branch_id)
    
    # Fetch the labels at those coordinates from the labels_out array
    branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
    
    # Find the unique label(s) associated with this branch
    unique_blob_labels = np.unique(branch_labels)
    
    # Store the unique label(s) in the branch_blob_mapping
    if len(unique_blob_labels) > 1:
        branch_blob_mapping[branch_id] = unique_blob_labels
    else:
        branch_blob_mapping[branch_id] = unique_blob_labels[0]
    
    # Step 3: Map each pixel in this branch to its corresponding blob label(s)
    for i, coord in enumerate(coordinates):
        pixel_blob_mapping[tuple(coord)] = branch_labels[i]


binary_volume_copy = np.copy(binary_volume)
while True:
    # Paso 4: Repetir los pasos 2 y 3 hasta que todos los blobs estén conectados
    binary_volume_copy = connect_blobs_3d(labels_out, pixel_blob_mapping, binary_volume_copy)
    
    # Free up memory by deleting unused variables
    del labels_out, pixel_blob_mapping, branch_blob_mapping
    
    labels_out, N = cc3d.connected_components(binary_volume_copy, return_N=True)
    print(f"Number of blobs: {N}")
    
    # Reinitialize pixel_blob_mapping to reduce memory usage
    # Initialize a dictionary to map each branch ID to its corresponding blob(s)
    branch_blob_mapping = {}
    pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

    skel = Skeleton(binary_volume_copy)
    branch_data = summarize(skel)

    # Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
    for branch_id in branch_data.index:
        # Get the coordinates of the current branch in the skeleton structure
        coordinates = skel.path_coordinates(branch_id)
        
        # Fetch the labels at those coordinates from the labels_out array
        branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
        
        # Find the unique label(s) associated with this branch
        unique_blob_labels = np.unique(branch_labels)
        
        # Store the unique label(s) in the branch_blob_mapping
        if len(unique_blob_labels) > 1:
            branch_blob_mapping[branch_id] = unique_blob_labels
        else:
            branch_blob_mapping[branch_id] = unique_blob_labels[0]
        
        # Step 3: Map each pixel in this branch to its corresponding blob label(s)
        for i, coord in enumerate(coordinates):
            pixel_blob_mapping[tuple(coord)] = branch_labels[i]
    
    if N == 3:
        print("All blobs are connected!")
        break

tiff.imwrite('C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/connected_blobs_TOP.tif', binary_volume_copy)


Number of LAbels: 29
Connecting blobs 21 and 23
Closest points: [390 111  32] in blob 21, [392 112  33] in blob 23
Number of blobs: 28
Connecting blobs 2 and 3
Closest points: [211 149 199] in blob 2, [211 147 199] in blob 3
Number of blobs: 27
Connecting blobs 10 and 13
Closest points: [321 194  16] in blob 10, [322 192  15] in blob 13
Number of blobs: 26
Connecting blobs 20 and 22
Closest points: [394  25 165] in blob 20, [395  23 165] in blob 22
Number of blobs: 25
Connecting blobs 10 and 12
Closest points: [321 183   8] in blob 10, [322 181   8] in blob 12
Number of blobs: 24
Connecting blobs 20 and 23
Closest points: [398   7  93] in blob 20, [398  20  93] in blob 23
Number of blobs: 23
Connecting blobs 1 and 2
Closest points: [193 171 191] in blob 1, [205 163 194] in blob 2
Number of blobs: 22
Connecting blobs 10 and 14
Closest points: [349  47 104] in blob 10, [351  45 104] in blob 14
Number of blobs: 21
Connecting blobs 18 and 20
Closest points: [398   9  93] in blob 18, [398  

## TOP - UNET

In [7]:
binary_volume = tiff.imread("C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/FINAL/skeleleton_clean_x20_th5_UNET_TOP.tif")

skel = Skeleton(binary_volume)
branch_data = summarize(skel)

labels_out = cc3d.connected_components(binary_volume)
labels_out, N = cc3d.connected_components(binary_volume, return_N=True)
print(f'Number of LAbels: {N}')

# Initialize a dictionary to map each branch ID to its corresponding blob(s)
branch_blob_mapping = {}
pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

# Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
for branch_id in branch_data.index:
    # Get the coordinates of the current branch in the skeleton structure
    coordinates = skel.path_coordinates(branch_id)
    
    # Fetch the labels at those coordinates from the labels_out array
    branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
    
    # Find the unique label(s) associated with this branch
    unique_blob_labels = np.unique(branch_labels)
    
    # Store the unique label(s) in the branch_blob_mapping
    if len(unique_blob_labels) > 1:
        branch_blob_mapping[branch_id] = unique_blob_labels
    else:
        branch_blob_mapping[branch_id] = unique_blob_labels[0]
    
    # Step 3: Map each pixel in this branch to its corresponding blob label(s)
    for i, coord in enumerate(coordinates):
        pixel_blob_mapping[tuple(coord)] = branch_labels[i]


binary_volume_copy = np.copy(binary_volume)
while True:
    # Paso 4: Repetir los pasos 2 y 3 hasta que todos los blobs estén conectados
    binary_volume_copy = connect_blobs_3d(labels_out, pixel_blob_mapping, binary_volume_copy)
    
    # Free up memory by deleting unused variables
    del labels_out, pixel_blob_mapping, branch_blob_mapping
    
    labels_out, N = cc3d.connected_components(binary_volume_copy, return_N=True)
    print(f"Number of blobs: {N}")
    
    # Reinitialize pixel_blob_mapping to reduce memory usage
    # Initialize a dictionary to map each branch ID to its corresponding blob(s)
    branch_blob_mapping = {}
    pixel_blob_mapping = np.zeros_like(labels_out, dtype=int)

    skel = Skeleton(binary_volume_copy)
    branch_data = summarize(skel)

    # Step 2: For each branch, get the coordinates and map them to the corresponding blob label(s)
    for branch_id in branch_data.index:
        # Get the coordinates of the current branch in the skeleton structure
        coordinates = skel.path_coordinates(branch_id)
        
        # Fetch the labels at those coordinates from the labels_out array
        branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]
        
        # Find the unique label(s) associated with this branch
        unique_blob_labels = np.unique(branch_labels)
        
        # Store the unique label(s) in the branch_blob_mapping
        if len(unique_blob_labels) > 1:
            branch_blob_mapping[branch_id] = unique_blob_labels
        else:
            branch_blob_mapping[branch_id] = unique_blob_labels[0]
        
        # Step 3: Map each pixel in this branch to its corresponding blob label(s)
        for i, coord in enumerate(coordinates):
            pixel_blob_mapping[tuple(coord)] = branch_labels[i]
    
    if N == 14:
        print("All blobs are connected!")
        break

tiff.imwrite('C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/connected_blobs_UNET_TOP.tif', binary_volume_copy)


Number of LAbels: 104
Connecting blobs 63 and 64
Closest points: [328  73  74] in blob 63, [329  73  76] in blob 64
Number of blobs: 103
Connecting blobs 81 and 82
Closest points: [357 127  45] in blob 81, [360 127  45] in blob 82
Number of blobs: 102
Connecting blobs 27 and 28
Closest points: [262  88 192] in blob 27, [263  86 193] in blob 28
Number of blobs: 101
Connecting blobs 41 and 42
Closest points: [300 167 106] in blob 41, [300 169 103] in blob 42
Number of blobs: 100
Connecting blobs 91 and 92
Closest points: [381 113   6] in blob 91, [384 114   8] in blob 92
Number of blobs: 99
Connecting blobs 57 and 60
Closest points: [324  64  93] in blob 57, [327  65  94] in blob 60
Number of blobs: 98
Connecting blobs 80 and 84
Closest points: [366 123  45] in blob 80, [368 122  44] in blob 84
Number of blobs: 97
Connecting blobs 35 and 36
Closest points: [291 118 141] in blob 35, [293 118 142] in blob 36
Number of blobs: 96
Connecting blobs 81 and 85
Closest points: [367  55  75] in bl